In [ ]:
# Install necessary libraries
!pip install geopandas folium panel pyngrok
!pip install gradio
# Install necessary libraries
!pip install geopandas folium gradio
!pip install jupyter_bokeh
import geopandas as gpd
import pandas as pd
import folium
import panel as pn
from pyngrok import ngrok
import gradio as gr

# Activate Panel extension
pn.extension()

In [ ]:
from google.colab import files
import pandas as pd
import geopandas as gpd

# Upload files
print("Upload the GeoJSON and Crime Data CSV files:")
uploaded_geojson = files.upload()
uploaded_csv = files.upload()

# Load GeoJSON
geojson_file = list(uploaded_geojson.keys())[0]
geo_data = gpd.read_file(geojson_file)

# Load CSV for general crime data
csv_file = list(uploaded_csv.keys())[0]
crime_data = pd.read_csv(csv_file)

# Display the first few rows to confirm data load
print("GeoJSON Data:")
print(geo_data.head())
print("\nCrime Data:")
print(crime_data.head())

# Additional upload for Assault.csv
print("Upload the Assault CSV file:")
uploaded_assault = files.upload()

# Load the Assault CSV
assault_file = list(uploaded_assault.keys())[0]
assault = pd.read_csv(assault_file)

# Display the first few rows of the Assault data to confirm
print("\nAssault Data:")
print(assault.head())


Upload the GeoJSON and Crime Data CSV files:


Saving Neighbourhood data Geojson.geojson to Neighbourhood data Geojson.geojson


Saving Neighbourhood_Crime_Rates_Flattened.csv to Neighbourhood_Crime_Rates_Flattened (1).csv
GeoJSON Data:
   OBJECTID_1                  AREA_NAME  HOOD_ID  ASSAULT_2014  ASSAULT_2015  \
0           1  South Eglinton-Davisville      174            63            61   
1           2              North Toronto      173            45            52   
2           3         Dovercourt Village      172            56            57   
3           4   Junction-Wallace Emerson      171           154           157   
4           5         Yonge-Bay Corridor      170           394           524   

   ASSAULT_2016  ASSAULT_2017  ASSAULT_2018  ASSAULT_2019  ASSAULT_2020  ...  \
0            70            82            85            70            82  ...   
1            43            52            55            77            72  ...   
2            79            94            94            96            75  ...   
3           166           157           157           182           169  ...   
4    

Saving Assault.csv to Assault.csv

Assault Data:
   OBJECTID_1                  AREA_NAME  HOOD_ID  ASSAULT_2014  ASSAULT_2015  \
0           1  South Eglinton-Davisville      174            63            61   
1           2              North Toronto      173            45            52   
2           3         Dovercourt Village      172            56            57   
3           4   Junction-Wallace Emerson      171           154           157   
4           5         Yonge-Bay Corridor      170           394           524   

   ASSAULT_2016  ASSAULT_2017  ASSAULT_2018  ASSAULT_2019  ASSAULT_2020  ...  \
0            70            82            85            70            82  ...   
1            43            52            55            77            72  ...   
2            79            94            94            96            75  ...   
3           166           157           157           182           169  ...   
4           487           603           576           660       

In [ ]:
def create_geojson_map():
    """
    Generates a folium map based on the GeoJSON data.
    """
    # Center the map on the neighborhood centroids
    map_center = [
        geo_data.geometry.centroid.y.mean(),
        geo_data.geometry.centroid.x.mean()
    ]
    m = folium.Map(location=map_center, zoom_start=12)

    # Add the GeoJSON data to the map
    folium.GeoJson(
        geo_data.to_json(),
        name="Neighborhoods",
        tooltip=folium.GeoJsonTooltip(fields=["AREA_NAME"], aliases=["Neighborhood"])
    ).add_to(m)

    return m


In [ ]:
# Function to generate a top 10 list for actual vs. predicted
def get_top_10_actual_vs_predicted(crime_type):
    comparison = create_actual_vs_predicted_table(crime_type)
    top_10 = comparison.nlargest(10, "Actual")
    return top_10[["AREA_NAME", "Actual", "Predicted"]]

# Update the Gradio Dashboard
with gr.Blocks() as dashboard:
    gr.Markdown("# Crime Event Dashboard")
    gr.Markdown("### Explore crime data using predictions and visualizations.")

    with gr.Tabs() as tabs:
        # Map View Tab
        with gr.Tab("Map View"):
            year_dropdown = gr.Dropdown(choices=years, label="Select Year")
            crime_type_dropdown = gr.Dropdown(choices=crime_types, label="Select Crime Type")
            map_output = gr.HTML()
            top_10_output = gr.Dataframe(headers=["Neighborhood", "Crime Events"])
            generate_map_button = gr.Button("Generate Map and Top 10 Data")

            generate_map_button.click(
                fn=map_and_top_10_selector,
                inputs=[year_dropdown, crime_type_dropdown],
                outputs=[map_output, top_10_output]
            )

        # Actual vs Predicted Tab
        with gr.Tab("Actual vs. Predicted"):
            crime_type_dropdown = gr.Dropdown(choices=crime_types, label="Select Crime Type")
            top_10_table_output = gr.Dataframe(headers=["Neighborhood", "Actual", "Predicted"])
            generate_top_10_button = gr.Button("Generate Top 10 List")

            # Generate the top 10 actual vs. predicted table
            generate_top_10_button.click(
                fn=get_top_10_actual_vs_predicted,
                inputs=[crime_type_dropdown],
                outputs=top_10_table_output
            )

# Launch the dashboard
dashboard.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5244d5beb0425886c3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Display the first few rows of the assault DataFrame to confirm the data load
print("Assault Data:")
print(assault.head())

# Example: Descriptive statistics of the assault data
print("\nDescriptive Statistics:")
print(assault.describe())

# Example: Group data by AREA_NAME to get the sum of assaults over all years
assault_by_area = assault.groupby('AREA_NAME').sum()

# Display the aggregated data
print("\nTotal Assaults by Area:")
print(assault_by_area[['ASSAULT_2014', 'ASSAULT_2015', 'ASSAULT_2016',
                       'ASSAULT_2017', 'ASSAULT_2018']])


Assault Data:
   OBJECTID_1                  AREA_NAME  HOOD_ID  ASSAULT_2014  ASSAULT_2015  \
0           1  South Eglinton-Davisville      174            63            61   
1           2              North Toronto      173            45            52   
2           3         Dovercourt Village      172            56            57   
3           4   Junction-Wallace Emerson      171           154           157   
4           5         Yonge-Bay Corridor      170           394           524   

   ASSAULT_2016  ASSAULT_2017  ASSAULT_2018  ASSAULT_2019  ASSAULT_2020  ...  \
0            70            82            85            70            82  ...   
1            43            52            55            77            72  ...   
2            79            94            94            96            75  ...   
3           166           157           157           182           169  ...   
4           487           603           576           660           383  ...   

   ASSAULT_RATE_20

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Reload the Assault dataset
assault_file_path = "/mnt/data/Assault.csv"
assault = pd.read_csv(assault_file_path)

# Extract columns for predictions
def prepare_data_by_area(assault):
    year_columns = [col for col in assault.columns if col.startswith('ASSAULT_') and col.split('_')[-1].isdigit()]
    years = [int(col.split('_')[-1]) for col in year_columns if int(col.split('_')[-1]) < 2023]
    X = np.array(years).reshape(-1, 1)  # Years as features
    area_predictions = {}
    mse_scores = {}
    r2_scores = {}

    for index, row in assault.iterrows():
        y = row[[f"ASSAULT_{year}" for year in years]].values

        # Normalize target values
        scaler = StandardScaler()
        y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()

        # Split the data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

        # Train the model with optimized hyperparameters
        model = XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)
        model.fit(X_train, y_train)

        # Predict for 2023 and inverse the scaling
        prediction_scaled = model.predict(np.array([[2023]]))[0]
        prediction_2023 = scaler.inverse_transform([[prediction_scaled]])[0, 0]

        # Evaluate the model
        y_pred = scaler.inverse_transform(model.predict(X_test).reshape(-1, 1)).flatten()
        mse = mean_squared_error(scaler.inverse_transform(y_test.reshape(-1, 1)).flatten(), y_pred)
        r2 = r2_score(scaler.inverse_transform(y_test.reshape(-1, 1)).flatten(), y_pred)

        # Store predictions and scores
        area_predictions[row["AREA_NAME"]] = prediction_2023
        mse_scores[row["AREA_NAME"]] = mse
        r2_scores[row["AREA_NAME"]] = r2

    return area_predictions, mse_scores, r2_scores

# Run predictions
area_predictions, mse_scores, r2_scores = prepare_data_by_area(assault)

# Include Actual Data and Accuracy Scores
actual_column = "ASSAULT_2023"
if actual_column in assault.columns:
    results_df = pd.DataFrame({
        "Area": assault["AREA_NAME"],
        "Actual_Assaults_2023": assault[actual_column],
        "Predicted_Assaults_2023": assault["AREA_NAME"].map(area_predictions),
        "MSE": assault["AREA_NAME"].map(mse_scores),
        "R2_Score": assault["AREA_NAME"].map(r2_scores),
    })
    # Display the results
    print("\nActual vs. Predicted Assaults for 2023 with Accuracy Scores:")
    print(results_df)
else:
    print(f"\nColumn '{actual_column}' is not available in the dataset for actual 2023 data.")


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/Assault.csv'

In [ ]:
# Print actual values, predicted values, and accuracy scores

# Ensure the Assault DataFrame is properly loaded
actual_column = "ASSAULT_2023"

if actual_column in assault.columns:
    # Create a DataFrame for actual vs predicted
    results_df = pd.DataFrame({
        "Area": assault["AREA_NAME"],
        "Actual_Assaults_2023": assault[actual_column],
        "Predicted_Assaults_2023": assault["AREA_NAME"].map(area_predictions)
    })

    # Display accuracy scores (MSE and R2)
    mse_values = [mse_scores[area] for area in results_df["Area"]]
    r2_values = [r2_scores[area] for area in results_df["Area"]]
    results_df["MSE"] = mse_values
    results_df["R2_Score"] = r2_values

    # Print the results
    print("\nActual vs. Predicted Assaults for 2023 with Accuracy Scores:")
    print(results_df)
else:
    print(f"\nColumn '{actual_column}' is not available in the dataset for actual 2023 data.")



Actual vs. Predicted Assaults for 2023 with Accuracy Scores:
                          Area  Actual_Assaults_2023  Predicted_Assaults_2023  \
0    South Eglinton-Davisville                   101               280.157593   
1                North Toronto                   105               400.631958   
2           Dovercourt Village                   104               427.716766   
3     Junction-Wallace Emerson                   229               550.724548   
4           Yonge-Bay Corridor                   543              2114.691650   
..                         ...                   ...                      ...   
153     West Humber-Clairville                   322               603.410034   
154                Black Creek                   231               616.651855   
155       Pelmo Park-Humberlea                   104               480.859528   
156                 Humbermede                   159               559.236572   
157              Humber Summit                 